In [1]:
from collections import defaultdict

### An implementation of an NFA class that computes on a input strings

In [84]:
class NFA:
    def __init__(self, transition_file):
        self.transitions = defaultdict(frozenset) 
        self.alphabet = set()
        with open(transition_file, 'r') as file:
            line = '#'
            while line.startswith('#') or line=='':
                line = next(file).strip()
            
            self.start, acc = line.split(' ')
            self.accept = acc.split(',')
            for line in file:
                line = line.strip()
                if line.startswith('#') or line=='':
                    continue
                    
                parts = line.replace(" ","").split("->")
                state, symbol = parts[0].split(",")
                if symbol:
                    self.alphabet.add(symbol)
                self.transitions[(state,symbol)] = frozenset(parts[1].split(","))

        
    def __repr__(self):
        ret = 'Symbols: '+(",".join(list(self.alphabet)))
        ret += "\nStart state: " + self.start
        ret += "\nAccept state(s):" + ",".join(self.accept)
        ret += "\nTransitions:"
        for state_symbol in self.transitions:
            state, symbol = state_symbol
            end_states = self.transitions[state_symbol]
            ret+=f"\n{state_symbol} -> {end_states}"
            
        return ret
    
    def compute(self, string):
        #TODO: compute on the NFA, create a tree (similar to the one discussed)
        #in lectures and accept a string if upon consuming all the string,
        #one of the end states is an accept state, otherwise reject.
        #Using frozenset, a set that is hashable, may come in handy.
        
        state = frozenset([self.start])
        
        next_state_initial = [self.start]
        for s in state:
            for n in self.transitions[(s, '')]:
                next_state_initial.append(n)
        state = frozenset(next_state_initial)
        
        for x in string:
            next_state = []
            for s in state:
                for n in self.transitions[(s, x)]:
                    next_state.append(n)
                    for z in self.transitions[(n, '')]:
                        next_state.append(z)
            state = frozenset(next_state)
        
        result = 'reject'
        for s in state:
            if s in self.accept:
                result = 'accept'
        
        return result
    
        

### Creating a sample NFA that accepts $0^n$ and $(01)^n$

In [85]:
with open ('sample_transitions.txt', 'w') as file:
    file.write("""
#start_state accept_states
q0 q1,q2
q0, -> q1,q2
q1,0 -> q1
q2,0 -> q3
q3,1 -> q2
""")

In [86]:
nfa = NFA('./sample_transitions.txt')
print(nfa)

Symbols: 0,1
Start state: q0
Accept state(s):q1,q2
Transitions:
('q0', '') -> frozenset({'q2', 'q1'})
('q1', '0') -> frozenset({'q1'})
('q2', '0') -> frozenset({'q3'})
('q3', '1') -> frozenset({'q2'})


### Tests

In [87]:
#Your code should pass the following tests, and others if we add
#Also, please test your code on other NFAs, too make sure it is working 
#correctly.

# Given code, which is incorrect
# assert nfa.compute("", "accept"), "Empty string should be accepted"
# assert nfa.compute("0", "accept"), "0 should be accepted"
# assert nfa.compute("010", "reject"), "010 does not belong to the language"
# assert nfa.compute('01'*10, "accept"), "(01)^10 is part of the language."
# assert nfa.compute('0'*10, "accept"), "0^10 is part of the language."

# My corrected syntax tests, which check the same thing
assert nfa.compute("") == "accept", "Empty string should be accepted"
assert nfa.compute("0") == "accept", "0 should be accepted"
assert nfa.compute("010") == "reject", "010 does not belong to the language"
assert nfa.compute('01'*10) == "accept", "(01)^10 is part of the language."
assert nfa.compute('0'*10) == "accept", "0^10 is part of the language."

### Submissions

Please fillin the "compute" function and test your code with a number of differnt NFAs. Once you made sure it is bug free submit your jupyter notebook in Gradescope.

IMPORTANT: Please rename your notebook to your GTID.ipyn (e.g. hhassanzadeh3.ipynb) before submission.

In [88]:
# custom test case that comes from the book

import random

with open ('booktest3.txt', 'w') as file:
    file.write("""
    #start_state accept_states
    q1 q1
    q1,b -> q2
    q1, -> q3
    q2,a -> q2,q3
    q2,b -> q3
    q3,a -> q1
    """)

nfa = NFA('./booktest3.txt')
print(nfa)

assert nfa.compute("") == "accept", "Empty string should be accepted"
assert nfa.compute("a") == "accept", "a should be accepted"
assert nfa.compute("baba") == "accept", "baba should be accepted"
assert nfa.compute("baa") == "accept", "baa should be accepted"
assert nfa.compute("b") == "reject", "b should be rejected"
assert nfa.compute("bb") == "reject", "bb should be rejected"
assert nfa.compute("babba") == "reject", "babba should be rejected"

for _ in range(100):
    assert nfa.compute("b" * random.randint(1, 100)) == "reject", "strings of all b's should be rejected"

Symbols: a,b
Start state: q1
Accept state(s):q1
Transitions:
('q1', 'b') -> frozenset({'q2'})
('q1', '') -> frozenset({'q3'})
('q2', 'a') -> frozenset({'q3', 'q2'})
('q2', 'b') -> frozenset({'q3'})
('q3', 'a') -> frozenset({'q1'})
